https://www.maxlist.xyz/2018/08/25/python_scrapy_ptt/
https://bigdatafinance.tw/index.php/tech/coding/525-python-scrapy
https://www.digitalocean.com/community/tutorials/how-to-crawl-a-web-page-with-scrapy-and-python-3
https://medium.com/@peilee_98185/python-scrapy-%E5%88%9D%E6%8E%A2-1bbe3ebbb36

# 關於Scrapy
Scrapy是值得一推的，它是一個爬蟲的框架，而不是一個簡單的爬蟲。它整合了爬取，處理數據，存儲數據的一條龍服務。<br>
如果你只需要偶爾的一兩次爬爬網頁，前面的教程已經夠了，如果你需要每天靠爬蟲吃飯，Scrapy還是有必要了解的。<br>

# Scrapy的優勢 
Scrapy是一個整合了的爬蟲框架，有著非常健全的管理系統。而且它也是分佈式爬蟲，但是比我們之前寫的那個分佈式爬蟲高級多了。<br>
下面就是Scrapy的框架示意圖(來源)。它的管理體系非常複雜，但是特別高效，讓你又刷網頁，又下載，同時能處理數據<br>

![Image of Yaktocat](https://morvanzhou.github.io/static/results/scraping/5-2-2.png)

# 一、環境設置

### 0. 安裝Scrapy
在終端機執行:pip install scrapy

### 1. 建立一個Project
<mark>在終端機執行:scrapy startproject<mark>

In [ ]:
1.輸入cd指令切換到欲新增專案的工作目錄 #終端機指令cd+路徑 : 移動到指定的路徑
  C:\Users\JOE>cd \BigData\Ch08
2.輸入scrapy startproject + 專案名稱，建立全新Scrapy專案
  C:\BigData\Ch08>scrapy startproject Ch8_3

輸入後會自動建立一些文件和設定，資料結構如下：

In [ ]:
(PROJECT)
│ scrapy.cfg #基礎設置
│
└─ptt #自己取的專案名稱資料夾
    │ items.py #此檔案定義爬取資料的item項目
    │ middlewares.py #中間件定義
    │ pipelines.py #管道定義，用於抓取數據後的處理
    │ settings.py #全局設置：可設定專案的延遲時間和輸出方式等。
    │ __init__.py
    │
    ├─spiders
    │ │ ptt.py #爬蟲主體，定義如何抓取需要的數據
    │ │ __init__.py
    │ │
    │ └─__pycache__
    │ __init__.cpython-36.pyc
    │
    └─__pycache__
            settings.cpython-36.pyc
            __init__.cpython-36.pyc
            items.cpython-36.pyc

### 2.在Project中再新增Python爬蟲程式(爬蟲主體)
<mark>在終端機執行:scrapy genspider<mark>

In [ ]:
1.輸入cd指令切換至專案目錄  #終端機指令cd+路徑 : 移動到指定的路徑
  C:\BigData\Ch08>cd Ch8_3\spiders  #移動到spiders資料夾中
2.輸入scrapy genspider在Scrapy專案中再新增Python程式
  C:\BigData\Ch08\Ch8_3\spiders >scrapy genspider ptt ptt.cc
#在spiders資料夾中，建立名為ptt的程式ptt.py，目標是欲爬取的網域ptt.cc

In [ ]:
#以上步驟會自行建立pttnba.py檔案，內容自行生成以下:

import scrapy


class PttSpider(scrapy.Spider):
    name = 'ptt'
    allowed_domains = ['ptt.cc']
    start_urls = ['http://ptt.cc/']

    def parse(self, response):
        pass

# 二、撰寫Scrapy爬蟲

### 1.設置items.py
首先在items.py內定義一些想要抓取的項目，本次爬取項目包含PTT 文章標題、文章作者、發文日期、噓推文數、內文網址等等。

In [ ]:
import scrapy

class PostItem(scrapy.Item):
    title = scrapy.Field()    #以下為自行新增欲爬取的項目，這些屬性的值將在爬蟲主體中賦予
    author = scrapy.Field()
    date = scrapy.Field()
    push = scrapy.Field()
    url = scrapy.Field()

### 2.設置spiders \ ptt.py(爬蟲主體)
回到爬蟲主體，裡面包含三個屬性一個函數：

-> name：爬蟲程式的名稱，在Scrapy稱為蜘蛛Spider<br>
-> allowed_domains：定義允許爬取的網域，沒有定義則表示任何網域都可以爬取<br>
-> start_urls：開始爬取的網址，允許多個<br>
-> parse()函數：此函數是實際爬取資料的Python程式碼

In [ ]:
from ptt.items import PostItem
import scrapy
import time

class PTTSpider(scrapy.Spider):
    name = 'ptt'
    allowed_domains = ['ptt.cc']
    start_urls = ['https://www.ptt.cc/bbs/Gossiping/index.html']
    
    def parse(self, response):
    for i in range(100): #此次任務是要爬PTT Gossiping版100頁的內容，這邊url寫了一個迴圈來爬取頁面。
        time.sleep(1)
        url = "https://www.ptt.cc/bbs/Gossiping/index" + str(39164 - i) + ".html"
        yield scrapy.Request (url, cookies={'over18': '1'}, callback=self.parse_article)  #cookies={‘over18′:’1’}跳過判別年齡頁
        #yield是Python關鍵字，類似函數的return關鍵字可以回傳資料，只是傳回的是產生器，如同for迴圈的range()函數
        #因為parse()會依序傳回多個字典的爬取資料，所以是使用yield回傳，而不是return
    
    def parse_article(self, response):
    item = PostItem()
    target = response.css("div.r-ent")


    for tag in target:
        try:
            item['title'] = tag.css("div.title a::text")[0].extract()
            item['author'] = tag.css('div.author::text')[0].extract()
            item['date'] = tag.css('div.date::text')[0].extract()
            item['push'] = tag.css('span::text')[0].extract()
            item['url'] = tag.css('div.title a::attr(href)')[0].extract()

            yield item

        except IndexError:
            pass
        continue

scrapy可使用css和xpath来定位元素，它有五个基本方法：<br>

-> xpath()： 使用xpath語言選擇的節點<br>
-> css()： 使用css語言選擇的節點<br>
-> extract()： 返回被選擇元素的unicode字符串<br>
-> extract_first()：返回第一個匹配元素的unicode字符串<br>
-> re(): 返回通过正則表達式提取的unicode字符串列表<br>

In [ ]:
#https://bigdatafinance.tw/index.php/tech/coding/525-python-scrapy
#其他範例

import scrapy, json
from ..items import TuchongItem

class PhotoSpider(scrapy.Spider):
    name = 'photo'
    # allowed_domains = ['tuchong.com']
    # start_urls = ['http://tuchong.com/']

    def start_requests(self):
        url = 'https://tuchong.com/rest/tags/%s/posts?page=%d&count=20&order=weekly';
        # 抓取10個頁面，每頁20個圖集
        # 指定parse 作為回調函數並返回Requests 請求對象
        for page in range(1, 11):
            yield scrapy.Request(url=url % ('美女', page), callback=self.parse)

    # 回調函數，處理抓取內容填充TuchongItem 屬性
    def parse(self, response):
        body = json.loads(response.body_as_unicode())
        items = []
        for post in body['postList']:
            item = TuchongItem()
            item['type'] = post['type']
            item['post_id'] = post['post_id']
            item['site_id'] = post['site_id']
            item['title'] = post['title']
            item['url'] = post['url']
            item['excerpt'] = post['excerpt']
            item['image_count'] = int(post['image_count'])
            item['images'] = {}
            # 將images 處理成{img_id: img_url} 對像數組
            for img in post.get('images', ''):
                img_id = img['img_id']
                url = 'https://photo.tuchong.com/%s/f/%s.jpg' % (item['site_id'], img_id)
                item['images'][img_id] = url

            item['tags'] = []
            # 將tags 處理成tag_name 數組
            for tag in post.get('tags', ''):
                item['tags'].append(tag['tag_name'])
            items.append(item)
        return items

# 三、運行Scrapy爬蟲

<mark>返回終端機執行:scrapy crawl<mark>

In [ ]:
C:\BigData\Ch08\Ch8_3>scrapy crawl ptt

如果需要對抓取的結果進行保存，只需要在命令行加參數 -o {filename} 即可：

# 四、保存結果

In [ ]:
scrapy crawl ptt -o output.json # 輸出為JSON文件
scrapy crawl ptt -o output.csv # 輸出為CSV文件

# 五、處理下一頁的資料

In [ ]:
yield scrapy.Request(nextPg, callback=self.parse)
上述程式碼是建立Request物件的HTTP請求，第一個參數是網址，在callback參數指定剖析回應資料需呼叫的回撥函數，以此例是呼叫自己的parse()函數，
簡單說，這一列程式碼就是呼叫parse()函數繼續剖析下一頁HTTP請求的回應資料，直到沒有下一頁超連結為止

# 六、合併從多個頁面爬取的資料
前面可以爬取多頁資料，使用的是Request物件，<br>
response.follow( )函數可以建立更簡潔的方式，來處理下一頁的超連結資料物件，並且合併從多個頁面爬取的資料。

### 使用response.follow( )函數

In [ ]:
        nextPg = response.xpath("//li[@class='next']/a/@href").extract_first()
        if nextPg is not None:
            yield response.follow(nextPg, callback=self.parse)

# 七、最佳化Scrapy
由於Scrapy專案中，預設同步16個檔案下載和下載之間並沒有任何延遲，是非常快速的網頁瀏覽，<br>
但卻很容易讓Web伺服器偵測到是網路爬蟲，所以拒絕存取。<br>
為了最佳化爬蟲程式，建議在Scrapy專案的settings.py設定檔案指定同步下載的檔案數和下載檔案之間的延遲時間。

In [ ]:
CONCURRENT_REQUESTS_PER_DOMAIN = 1
DOWNLOAD_DELAY = 5

# 八、在專案中使用Item和Item Pipeline
https://ithelp.ithome.com.tw/articles/10206612?sc=iThelpR<br>
在item中定義需要儲存的內容，然後在pipeline處理item

### 1. Item
Scrapy的爬取結果可以使用字典或Item項目物件，基礎的例子其實都用字典，對初學者來說已經足夠，<br>
但對大型爬蟲程式來說，建議使用Item項目儲存爬取資料，以便使用Item Pipeline項目管道來清理、驗證和處理取得的資料。<br>
使用Python的字典方式存資料，也可能會有缺點，字典雖然方便卻缺少結構性，<br>
容易打錯字或者回傳不一致的數據，特別是在多個Spider的專案中。<br>

上面的例子皆已使用Item項目，而非字典，主要步驟：<br>
1. 到items.py定義想要抓取的項目
2. 在最上面新增一行，import我們item.py的class： from ..items import TuchongItem
3. 修改def parse(self, response)的內容為Item項目

### 2. Item Pipeline
spider每抓取到一個(item)項目後，便會送到Item Pipeline，經過多個元件依序串起來成為一個資料處理的管線。<br>
可應用在:清洗資料、驗證資料、過濾重複資料、資料存入資料庫<br>

In [ ]:
#自行生成的pipeline.py檔案內容
class RickyTestPipeline(object):
    def process_item(self, item, spider):
        return item

來看看怎麼使用吧：<br>
將原本推文數量從字串轉為整數：<br>

In [ ]:
class MyfirstscrapyprojectPipeline(object):
    def process_item(self, item, spider):
        item['push'] = int(item['push'])
        return item

執行之後，會發現沒有變化還是字串啊？因為還沒結束！<br>

要啟用pipeline，打開settings.py這隻檔案，找到ITEM_PIPELINES（如下程式碼）部分將註解(#)拿掉：

In [ ]:
#ITEM_PIPELINES = {
#    'ricky_test.pipelines.RickyTestPipeline': 300,
#}

就可以發現終端輸出的內容推文次數為整數了！<br>
這樣的方式，讓我們可以只選擇啟用特定的Item Pipeline。<br>

後面的數字300表示Pipeline的執行順序，小的會先執行。

# 九、輸出Scrapy爬取的資料
前面是用scrapy crawl指令來輸出爬取資料，<br>
實務上，我們可以直接在settings.py檔案設定專案的輸出方式。

In [ ]:
# 輸出 JSON 資料
FEED_FORMAT = "json"  #輸出格式，可以是json/csv/xml
FEED_URI = "pttnba.json"  #輸出檔名
FEED_EXPORT_ENCODING = "utf-8"  #指定使用的編碼

In [ ]:
再設定的Scrapy專案中輸出格式的編碼是"utf-8"時，在終端機執行scrapy crawl時就不需要指定"-o"